In [100]:
import requests
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}

In [101]:
stock_id = 6257

## Crawl ROE

In [102]:
# crawl ROE
res = requests.get('https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID={}'.format(stock_id), headers = headers)
res.encoding = 'utf-8'
res.text

'\r\n<!doctype html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html lang="zh-Hant">\r\n<head>\r\n\r\n<meta charset="UTF-8">\r\n<meta name="viewport" content="width=device-width">\r\n<meta name="description" content="(6257) 矽格歷年財報評比,股票成交價格,成交量,獲利能力(如:營業收入,淨利,營業毛利率,淨利率),股東權益報酬率ROE,資產報酬率ROA,每股盈餘EPS,每股淨值BPS,現金股利,股票股利,殖利率 等交易資料統計">\r\n<title>(6257) 矽格歷年經營績效統計 - 合併財報 - Goodinfo!台灣股市資訊網</title>\r\n\r\n<link rel="canonical" href="https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=6257&RPT_CAT=M%5FYEAR"/>\r\n<link rel="icon" href="image/Goodinfo.ico">\r\n<link rel="stylesheet" type="text/css" href="../Library/css/normalize201_UTF-8.css"/>\r\n<link rel="stylesheet" type="text/css" href="../Library/css/Public_UTF-8.css"/>\r\n<link rel="stylesheet" type="text/css" href="../Library/jsp/JSCal2-1.8/src/css/jscal2.css" />\r\n<link rel="stylesheet" type="text/css" href="../Library/jsp/JSCal2-1.8/src/css/border-radius.css" />

In [103]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')
data = soup.select_one('#txtFinDetailData')

In [104]:
import pandas
dfs = pandas.read_html(data.prettify())
df = dfs[1]
df.columns = df.columns.get_level_values(3)

In [105]:
df.head()

,年度,股本 (億),財報 評分,收盤,平均,漲跌,漲跌 (%),營業 收入,營業 毛利,營業 利益,...,稅後 淨利,營業 毛利,營業 利益,業外 損益,稅後 淨利,ROE (%),ROA (%),稅後 EPS,年增 (元),BPS (元)
0,20Q2,42.2,62,38.8,37.9,+1.7,+4.6,56.2,15.5,9.66,...,7.97,27.5,17.2,0.3,14.8,12.4 (年估),6.46 (年估),1.89,+0.67,26.25
1,2019,42.1,62,37.1,31.5,+10.05,+37.2,100,30.4,20.7,...,12.9,30.3,20.6,-0.01,15.6,12.3,7.07,3.26,+0.25,26.75
2,2018,39,64,27.05,34.3,-5.9,-17.9,95.4,26.6,17.3,...,11.6,27.8,18.1,0.7,14.1,12.1,7.07,3.01,+0.25,25.2
3,2017,36,60,32.95,27.3,+9.65,+41.4,68.3,18.7,16.1,...,9.89,27.4,23.6,-0.05,17.5,13,7.43,2.76,+0.59,23.49
4,2016,36,62,23.3,23.4,+1.05,+4.7,58,16.1,9.6,...,7.76,27.8,16.6,0.56,13.8,10,6.17,2.17,+0.06,22.34


In [106]:
df.columns

Index(['年度', '股本  (億)', '財報  評分', '收盤', '平均', '漲跌', '漲跌  (%)', '營業  收入',
       '營業  毛利', '營業  利益', '業外  損益', '稅後  淨利', '營業  毛利', '營業  利益', '業外  損益',
       '稅後  淨利', 'ROE  (%)', 'ROA  (%)', '稅後  EPS', '年增  (元)', 'BPS  (元)'],
      dtype='object')

In [107]:
import re
ROEs_vals = df["ROE  (%)"].values

years = 5
ROEs = []
for ROE in ROEs_vals[:years]:
    ROE = re.findall(r"[-+]?\d*\.\d+|\d+", ROE)
    if len(ROE) > 1: 
        print ("ERROR: String parse error!")
    if len(ROE) == 0:
        print ("ERROR: No ROE record for some years!")
        
    ROE = float(ROE[0])
    ROEs.append(ROE)

print (ROEs)

[12.4, 12.3, 12.1, 13.0, 10.0]


In [108]:
import re
EPSs_vals = df["稅後  EPS"].values

years = 5
EPSs = []
for EPS in EPSs_vals[:years]:
    EPS = re.findall(r"[-+]?\d*\.\d+|\d+", EPS)
    if len(EPS) > 1: 
        print ("ERROR: String parse error!")
    if len(EPS) == 0:
        print ("ERROR: No EPS record for some years!")
        
    EPS = float(EPS[0])
    EPSs.append(EPS)

print (EPSs)

[1.89, 3.26, 3.01, 2.76, 2.17]


## 股利政策

In [136]:
import requests
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}
res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
res.encoding = 'utf-8'
#res.text

In [142]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')
data = soup.select_one("table", class_="tb-outline")

import pandas
dfs = pandas.read_html(data.prettify())

df = dfs[0]
df = df[1:]
df.head()

,所屬年度,發放年度,除權日,除息日,除權息 前股價,股票股利,現金股利,EPS,配息率,現金 殖利率,扣抵稅率,增資 配股率,增資 認購價
1,2019後半,2020.0,NaN,07/27,46.40,0.0,2.20,3.26,67.48%,4.74%,NaN,0.0,0.0
2,2018,2019.0,NaN,07/15,31.25,0.0,2.06,3.01,68.44%,6.59%,NaN,0.0,0.0
3,2017,2018.0,NaN,07/09,36.95,0.0,1.76,2.76,63.77%,4.77%,NaN,0.0,0.0
4,2016,2017.0,NaN,08/07,28.50,0.0,1.65,2.17,76.04%,5.78%,15.66%,0.0,0.0
5,2015,2016.0,NaN,07/28,24.65,0.0,1.60,2.11,75.83%,6.49%,17.49%,0.0,0.0


In [161]:
year_to_check = 5
cash_dividend_payouts = df["現金股利"][:year_to_check].astype('float')
EPSs = df["EPS"][:year_to_check].astype('float')

In [163]:
cash_dividend_ratios = cash_dividend_payouts / EPSs

In [164]:
cash_dividend_ratios

1    0.674847
2    0.684385
3    0.637681
4    0.760369
5    0.758294
dtype: float64

## 每股淨值

In [165]:
import requests
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}
res = requests.get("https://histock.tw/stock/{}/%E6%AF%8F%E8%82%A1%E6%B7%A8%E5%80%BC".format(stock_id), headers = headers)
res.encoding = 'utf-8'
#res.text

In [167]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')
data = soup.select_one("table", class_="tb-outline")

import pandas
dfs = pandas.read_html(data.prettify())

df = dfs[0]
df.head()

,季別/年度,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Q1,20.03,20.93,21.51,22.38,22.20,22.75,24.28,25.90,26.25
1,Q2,19.07,19.48,20.59,20.81,21.10,21.78,23.74,24.65,26.25
2,Q3,19.78,20.21,21.38,21.39,21.71,22.40,24.58,25.66,-
3,Q4,20.46,20.86,21.95,21.86,22.34,23.49,25.20,26.75,-


In [180]:
last_year = "2019"
prev_net_val_per_share = df.iloc[3][last_year]
prev_net_val_per_share

26.75

## 本益比

In [ ]:
import requests
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}
res = requests.get("https://histock.tw/stock/{}/%E6%9C%AC%E7%9B%8A%E6%AF%94".format(stock_id), headers = headers)
res.encoding = 'utf-8'
#res.text